# Generating HTML knowledge graph

In [10]:
import os
import pickle
from pyvis.network import Network
from dotenv import load_dotenv

# Define function to load KB from .pkl file
def load_kb(filename):
    """Load a knowledge base (KB) from a .pkl file."""
    with open(filename, "rb") as f:
        kb = pickle.load(f)
    return kb



class KB:
    def __init__(self):
        self.entities = {}
        self.relations = []
        self.sources = {}

    def merge_with_kb(self, kb2):
        for r in kb2.relations:
            self.add_relation(r)

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r2):
        r1 = [r for r in self.relations if self.are_relations_equal(r2, r)][0]
        spans_to_add = [span for span in r2["meta"]["spans"]
                        if span not in r1["meta"]["spans"]]
        r1["meta"]["spans"] += spans_to_add

    def add_entity(self, e):
        self.entities[e["title"]] = {k: v for k, v in e.items() if k != "title"}

    def add_relation(self, r):
        candidate_entities = [r["head"], r["tail"]]
        entities = [{"title": ent} for ent in candidate_entities]

        for e in entities:
            self.add_entity(e)

        r["head"] = entities[0]["title"]
        r["tail"] = entities[1]["title"]

        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def print(self):
        print("Entities:")
        for e in self.entities.items():
            print(f"  {e}")
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")
        print("Sources:")
        for s in self.sources.items():
            print(f"  {s}")



In [2]:
for i in range(1, 9):
    #load saved Named Entities and Relations (in kb file) from the KG_Construction.ipynb Jupyter Notebook
    kb = load_kb(f"Saved_Knowledge/Marie Curie {i}.pkl")

    #construct graph from the saved Named Entities and Relations (in kb file)
    net = Network(
        directed=True, 
        width="1200px", 
        height="1000px", 
        bgcolor="#FFFFFF",
        notebook=True,
        )

    # nodes
    color_entity = "#00FF00"
    for e in kb.entities:
        #net.add_node(e, shape="circle", color=color_entity)
        net.add_node(e)
        print("add note",e)

    #edges
    for r in kb.relations:
        net.add_edge(r["head"], r["tail"], title=r["type"], label=r["type"])
        #net.add_edge(r["head"], r["tail"], label=r["type"])
        print("add relation",r["head"]," ",r["tail"])

    net.repulsion(
        node_distance=230,
        #central_gravity=0.02,
        #spring_length=200,
        #spring_strength=0.05,
        damping=0.01
        )
    net.set_edge_smooth('dynamic')
    net.show(f'Knowledge Graph/Marie Curie {i}.html') 

add note Marie Curie
add note Pierre Curie
add note Kingdom of Poland
add note Warsaw
add note Russian Empire
add note Paris
add note Henri Becquerel
add note Paris street accident
add note polonium
add note radium
add note chemical element
add note Passy
add note Haute-Savoie
add note France
add note International Year of Chemistry
add note 2011
add note Marie Curie Skłodowska
add note 7 November 1867
add note Bronisława, née Boguska
add note Władysław Skłodowski
add note Congress Poland
add note January Uprising
add note 1863
add note Bolesław Prus
add note Lublin
add note mathematics
add note physics
add note Zofia
add note Bronisława
add note atheist
add note Flying University
add relation Marie Curie   Pierre Curie
add relation Pierre Curie   Marie Curie
add relation Kingdom of Poland   Warsaw
add relation Kingdom of Poland   Russian Empire
add relation Kingdom of Poland   Russian Empire
add relation Pierre Curie   Paris
add relation Henri Becquerel   Paris
add relation Pierre Cur

In [3]:
for i in range(1, 9):
    #load saved Named Entities and Relations (in kb file) from the KG_Construction.ipynb Jupyter Notebook
    kb = load_kb(f"Saved_Knowledge/Marie Curie_Extended {i}.pkl")

    #construct graph from the saved Named Entities and Relations (in kb file)
    net = Network(
        directed=True, 
        width="1200px", 
        height="1000px", 
        bgcolor="#FFFFFF",
        notebook=True,
        )

    # nodes
    color_entity = "#00FF00"
    for e in kb.entities:
        #net.add_node(e, shape="circle", color=color_entity)
        net.add_node(e)
        print("add note",e)

    #edges
    for r in kb.relations:
        net.add_edge(r["head"], r["tail"], title=r["type"], label=r["type"])
        #net.add_edge(r["head"], r["tail"], label=r["type"])
        print("add relation",r["head"]," ",r["tail"])

    net.repulsion(
        node_distance=230,
        #central_gravity=0.02,
        #spring_length=200,
        #spring_strength=0.05,
        damping=0.01
        )
    net.set_edge_smooth('dynamic')
    net.show(f'Knowledge Graph/Marie Curie_Extended {i}.html') 

add note Marie Curie
add note Pierre Curie
add note Kingdom of Poland
add note Warsaw
add note Russian Empire
add note Paris
add note Henri Becquerel
add note Paris street accident
add note polonium
add note radium
add note chemical element
add note Passy
add note Haute-Savoie
add note France
add note International Year of Chemistry
add note 2011
add note Marie Curie Skłodowska
add note 7 November 1867
add note Bronisława, née Boguska
add note Władysław Skłodowski
add note Congress Poland
add note January Uprising
add note 1863
add note Bolesław Prus
add note Lublin
add note mathematics
add note physics
add note Zofia
add note Bronisława
add note atheist
add note Flying University
add note grandfather
add note Curie
add note mother
add note sibling
add note father
add relation Marie Curie   Pierre Curie
add relation Pierre Curie   Marie Curie
add relation Kingdom of Poland   Warsaw
add relation Kingdom of Poland   Russian Empire
add relation Kingdom of Poland   Russian Empire
add relat

In [4]:
kb_unified = KB()  # Create a single KB instance

# Loop through files and merge their knowledge bases into the unified KB
for i in range(1, 9):
    kb = load_kb(f"Saved_Knowledge/Marie Curie_Extended {i}.pkl")
    kb_unified.merge_with_kb(kb)  # Merge each loaded KB into the unified KB

# Generate the network visualization
net = Network(
    directed=True, 
    width="1200px", 
    height="1000px", 
    bgcolor="#FFFFFF",
    notebook=True,
)

# Nodes
color_entity = "#00FF00"
for e in kb_unified.entities:
    net.add_node(e, shape="circle", color=color_entity)  # Use shape and color for clarity
    print("Added node:", e)

# Edges
for r in kb_unified.relations:
    net.add_edge(r["head"], r["tail"], title=r["type"], label=r["type"])
    print("Added relation:", r["head"], "->", r["tail"])

# Set network properties and display
net.repulsion(
    node_distance=230,
    damping=0.01
)
net.set_edge_smooth('dynamic')
net.show('Knowledge Graph/Marie Curie Unified.html') 

Added node: Marie Curie
Added node: Pierre Curie
Added node: Kingdom of Poland
Added node: Warsaw
Added node: Russian Empire
Added node: Paris
Added node: Henri Becquerel
Added node: Paris street accident
Added node: polonium
Added node: radium
Added node: chemical element
Added node: Passy
Added node: Haute-Savoie
Added node: France
Added node: International Year of Chemistry
Added node: 2011
Added node: Marie Curie Skłodowska
Added node: 7 November 1867
Added node: Bronisława, née Boguska
Added node: Władysław Skłodowski
Added node: Congress Poland
Added node: January Uprising
Added node: 1863
Added node: Bolesław Prus
Added node: Lublin
Added node: mathematics
Added node: physics
Added node: Zofia
Added node: Bronisława
Added node: atheist
Added node: Flying University
Added node: grandfather
Added node: Curie
Added node: mother
Added node: sibling
Added node: father
Added node: Kazimierz Żorie
Added node: Żorawskis
Added node: Szczuki
Added node: Kazimierz Żorąskis
Added node: Kazi

In [5]:
from neo4j import GraphDatabase

def save_kb_to_neo4j(kb, uri, username, password):
    # Connect to Neo4j database
    driver = GraphDatabase.driver(uri, auth=(username, password))
    with driver.session() as session:
        # Create entities
        for entity_title, entity_data in kb.entities.items():
            entity = {
                "title": entity_title,
                "url": entity_data.get("url", ""),
                "summary": entity_data.get("summary", "")
            }
            session.run(
                "MERGE (e:Entity {title: $title}) "
                "SET e.url = $url, e.summary = $summary",
                title=entity["title"],
                url=entity["url"],
                summary=entity["summary"]
            )

        # Create relations
        for relation in kb.relations:
            head = relation["head"]
            tail = relation["tail"]
            relation_type = relation["type"] if relation["type"] else "RELATION"  # Default type
            
            # Add or update relation in Neo4j
            result = session.run(
                "MATCH (h:Entity {title: $head}), (t:Entity {title: $tail}) "
                "MERGE (h)-[r:RELATION {type: $type}]->(t) "
                "SET r.type = $type RETURN h, t, r",
                head=head,
                tail=tail,
                type=relation_type
            )
            
            # Print detailed information about the relation
            for record in result:
                print(f"Added/Updated relation: {record['h']['title']} -[{relation_type}]-> {record['t']['title']}")

    # Close the connection
    driver.close()
    print("KB saved to Neo4j successfully.")


In [11]:
load_dotenv()

uri = os.getenv('NEO4J_URI')
neo4j_username = os.getenv('NEO4J_USERNAME')
neo4j_password = os.getenv('NEO4J_PASSWORD')

# Save KB to Neo4j
save_kb_to_neo4j(kb_unified, uri, neo4j_username, neo4j_password)

Added/Updated relation: Marie Curie -[spouse]-> Pierre Curie
Added/Updated relation: Pierre Curie -[spouse]-> Marie Curie
Added/Updated relation: Kingdom of Poland -[capital]-> Warsaw
Added/Updated relation: Kingdom of Poland -[country]-> Russian Empire
Added/Updated relation: Kingdom of Poland -[located in the administrative territorial entity]-> Russian Empire
Added/Updated relation: Pierre Curie -[place of death]-> Paris
Added/Updated relation: Henri Becquerel -[place of death]-> Paris
Added/Updated relation: Pierre Curie -[place of death]-> Paris street accident
Added/Updated relation: Henri Becquerel -[place of death]-> Paris street accident
Added/Updated relation: polonium -[discoverer or inventor]-> Marie Curie
Added/Updated relation: radium -[discoverer or inventor]-> Marie Curie
Added/Updated relation: polonium -[instance of]-> chemical element
Added/Updated relation: Marie Curie -[work location]-> Paris
Added/Updated relation: Passy -[located in the administrative territorial